In [30]:
from processing import preprocessing,get_movies_aspect_matrix,dict_movie_aspect,viewed_matrix
import json
import pandas as pd
import numpy as np
import scipy
import sklearn.preprocessing as pp
from compute_strength import film_strength
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

In [4]:
trainset = pd.read_csv('train.csv')
testset = pd.read_csv('test.csv')
def traite_train_test(df):
    df['actors'] = df['actors'].apply(lambda x: json.loads(x))
    df['director'] = df['director'].apply(lambda x: json.loads(x))
    df['genre'] = df['genre'].apply(lambda x: json.loads(x))
    return df
hehe_test = traite_train_test(trainset)
df_empty = traite_train_test(testset)
df_empty['user_id'] = df_empty['user_id'].astype('int')
df_empty['user_rating'] = df_empty['user_rating'].astype('float')
df_empty['movie'] = df_empty['movie'].astype('int')
hehe_test.index = range(len(hehe_test))
df_empty.index = range(len(df_empty))

In [5]:
df_all = hehe_test.append(df_empty)

<ipython-input-5-e39932f4636d>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = hehe_test.append(df_empty)


In [6]:
df_sum = df_all.copy()
movie_set = set(df_sum['movie'])
movie = df_sum[['movie','actors','director','genre']].loc[df_sum[['movie','actors','director','genre']].astype(str).drop_duplicates().index]
movie.index = range(len(movie))
movie = movie[['movie','actors','director','genre']].loc[movie[['movie','actors','director','genre']].astype(str).drop_duplicates().index]

In [7]:
ratings_dict = dict()
train_ratings_dict = dict()

train_ratings_dict["userID"] = []
train_ratings_dict["itemID"] = []
train_ratings_dict["rating"] = []


test_eva_dict = hehe_test.to_dict('records')
for row in test_eva_dict[:]:
    tuple_key = (row['user_id'],row['movie'])
    ratings_dict[tuple_key] = row['user_rating']
    train_ratings_dict["userID"].append(row['user_id'])
    train_ratings_dict["itemID"].append(row['movie'])
    train_ratings_dict["rating"].append(row['user_rating'])

In [8]:
compressed_test_ratings_dict = dict()
test_eva_dict = df_empty.to_dict('records')
for row in test_eva_dict[:]:
    if row['user_id'] not in compressed_test_ratings_dict:
        compressed_test_ratings_dict[row['user_id']] = []
    compressed_test_ratings_dict[row['user_id']].append((row['movie'],row['user_rating']))

In [9]:
films = dict()


test_eva_dict = movie.to_dict('records')
for row in test_eva_dict[:]:
    if row['movie'] not in films:
        films[row['movie']] = dict()
    films[row['movie']]['director'] = row['director']
    films[row['movie']]['genre'] = row['genre']
    films[row['movie']]['actors'] = row['actors']
    
    
    

In [10]:
movies_watched = viewed_matrix(train_ratings_dict, films, 'netflix')
movies_watched = pd.DataFrame.from_dict(
    movies_watched, dtype='int64', orient='index').T
movies_watched = movies_watched.replace(np.nan, 0)

<ipython-input-10-7b9550262de4>:2: FutureWarning: In a future version, passing float-dtype values containing NaN and an integer dtype will raise IntCastingNaNError (subclass of ValueError) instead of silently ignoring the passed dtype. To retain the old behavior, call Series(arr) or DataFrame(arr) without passing a dtype.
  movies_watched = pd.DataFrame.from_dict(


In [11]:
user_ids_in_matrix = movies_watched.index.values
movies_watched = movies_watched.T
movies_watched = scipy.sparse.csc_matrix(movies_watched.values)
normalized_matrix_by_column = pp.normalize(
    movies_watched.tocsc(), norm='l2', axis=0)
cosine_sims = normalized_matrix_by_column.T * normalized_matrix_by_column
assert cosine_sims.shape[0] == cosine_sims.shape[1] == len(
    user_ids_in_matrix)

In [12]:
sims = dict()
for i in user_ids_in_matrix:
    sims[i] = []
cosine_sims = cosine_sims.todok().items()

for ((row, col), sim) in cosine_sims:
    if row != col:
        sims[user_ids_in_matrix[row]].append(
                (user_ids_in_matrix[col], sim))

In [13]:
genres_in_db, movies_all_genres_matrix = get_movies_aspect_matrix(
    films, "genre")
directors_in_db, movies_all_directors_matrix = get_movies_aspect_matrix(
    films, "director")
actors_in_db, movies_all_actors_matrix = get_movies_aspect_matrix(
    films, "actors")
print('We have %d total aspects' %
      (len(genres_in_db)+len(directors_in_db)+len(actors_in_db)))

We have 20 genre (an example is 1134)


/home/jinfeng/Downloads/doctorant/Conférences/machine learning/data_and_codes/NETFLIX/processing.py:70: FutureWarning: In a future version, passing float-dtype values containing NaN and an integer dtype will raise IntCastingNaNError (subclass of ValueError) instead of silently ignoring the passed dtype. To retain the old behavior, call Series(arr) or DataFrame(arr) without passing a dtype.
  movies_all_aspects_matrix = pd.DataFrame.from_dict(aspects_associated_to_movies, dtype='int64', orient='index')


We have 389 director (an example is 745)
We have 745 actors (an example is 0)
We have 1154 total aspects


In [50]:
MUR = 0.1
MUG = 0.8
MUA = 0.1
MUD = 0.8

In [55]:
def dic_order_value_and_get_key(dicts, count):
    final_result = []
    sorted_dic = sorted([(k, v) for k, v in dicts.items()], reverse=True)
    tmp_set = set()
    for item in sorted_dic:
        tmp_set.add(item[1])
    for list_item in sorted(tmp_set, reverse=True)[:count]:
        for dic_item in sorted_dic:
            if dic_item[1] == list_item:
                final_result.append(dic_item[0])
    return final_result

In [57]:
user_item = {}
user_item_neg = {}

for user in set(df_all['user_id']):
    if user not in user_item:
        mean_rating = hehe_test[hehe_test['user_id'] == user]['user_rating'].mean()
        pos_df = hehe_test[hehe_test['user_id'] == user]
        pos_df = pos_df[pos_df['user_rating'] > mean_rating]
        user_item[user] = set(pos_df['user_id'].values)
for user,pos_item_set in user_item.items():
    unwatched_set = movie_set - pos_item_set
    if user not in user_item_neg:
        user_item_neg[user] = set()
    user_item_neg[user] = set(np.random.choice(list(unwatched_set), size=100, replace=False))
    

In [68]:
num_i = 101
def get_prediction_list(user, target,k):
    
    item_neg = user_item_neg[user]
    test_ratings_dict = dict()
    test_ratings_dict[user] = []
    items_list = []
    items_list.append(target)
    test_ratings_dict[user].append((target,1))
    for item in user_item_neg:
        test_ratings_dict[user].append((item,1))
        items_list.append(item)
    predictions = []
    for user_id, true_ratings in compressed_test_ratings_dict.items():
        if true_ratings:
            for (film_id, str_rating) in true_ratings:
                strength = film_strength(MUR, MUG, MUA, MUD, user_id, film_id, films, ratings_dict,
                                     sims[user_id], movies_all_genres_matrix, movies_all_directors_matrix, movies_all_actors_matrix)
                predictions.append((int(str_rating), strength))
    
    predicted_ratings = [round(y) for (x, y) in predictions]
    

    prediction_dict = dict(zip(items_list, predicted_ratings))
    top_k = dic_order_value_and_get_key(prediction_dict,k)
    return top_k,prediction_dict

In [69]:
import math
def hr_ndcg():
    hr1 = 0.0
    ndcg1 = 0.0
    hr2 = 0.0
    ndcg2 = 0.0
    test_eva_dict = df_empty.to_dict('records')
    for row in test_eva_dict[:]:
        user = row['user_id']
        target = row['movie']
        
        
        recommended_item_10,_ = get_prediction_list(user,target,10)
        recommended_item_20,_ = get_prediction_list(user,target,20)
        if target in recommended_item_10:
            hr1 +=1
            posi = recommended_item_10.index(target)
            ndcg1 += 1 / math.log(posi + 2,2)
        if target in recommended_item_20:
            hr2 +=1
            posi = recommended_item_20.index(target)
            ndcg2 += 1 / math.log(posi + 2,2)
    return hr1 / data.shape[0], ndcg1 / data.shape[0],hr2 / data.shape[0], ndcg2 / data.shape[0]
        

In [ ]:
hr_ndcg()

In [51]:
# predictions = []
# for user_id, true_ratings in compressed_test_ratings_dict.items():
#     if true_ratings:
#         for (film_id, str_rating) in true_ratings:
#             strength = film_strength(MUR, MUG, MUA, MUD, user_id, film_id, films, ratings_dict,
#                                      sims[user_id], movies_all_genres_matrix, movies_all_directors_matrix, movies_all_actors_matrix)
   
#             predictions.append((int(str_rating), strength))



In [52]:
# true_ratings = [x for (x, y) in predictions]
# predicted_ratings = [round(y) for (x, y) in predictions]

In [63]:
# sqrt(mean_squared_error(true_ratings, predicted_ratings)), mean_absolute_error(true_ratings, predicted_ratings)